# Road Following by Regression - Data Collection



## Introduction

We will try to make the JetBot to follow a desired road by using a regression model.

This is the notebook for data collection.

## Live Camera Feed
Let's create a camera instance and an image widget, then make a link between them.

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

size=224
camera = Camera.instance(width=size, height=size)
image = widgets.Image(format='jpeg', width=size, height=size)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Make sure the camera is working, then stop it for now.

In [2]:
camera.stop()

## Robot Instance
Let's create a robot instance for driving the JetBot motors.

In [3]:
from jetbot import Robot
robot = Robot()

Make sure the motors are working.

In [4]:
import time
robot.set_motors(-0.1, 0.1)
time.sleep(0.1)
robot.stop()

## Data Directory
Create a directory named `dataset_reg` for collecting the image data.

In [5]:
import os
DATASET_DIR = 'dataset_reg'
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directory was not created because they already exist')

Directory was not created because they already exist


## Define move_robot function
Define a function to control the motors depending on the x and y coordinates of the target point.

In [6]:
def move_robot(x, y):
    # move robot
    speed_gain = 0.3
    b = 0.3               # kind of steering gain
    x = x/size-0.5
    y = 1-y/size
    a = np.sqrt(x**2 + y**2)
    angle = np.pi/2-np.arctan2(y, x)+1E-6
    c = 0.5*a/np.sin(np.abs(angle))

    if angle >= 0:
        left_motor = speed_gain*(c+b)*np.abs(2*angle)
        right_motor = speed_gain*(c-b)*np.abs(2*angle)  
    else:
        left_motor = speed_gain*(c-b)*np.abs(np.abs(2*angle))
        right_motor = speed_gain*(c+b)*np.abs(np.abs(2*angle))

    left_motor = max(0, left_motor)
    right_motor = max(0, right_motor)              

    robot.set_motors(left_motor/2, right_motor/2)
    time.sleep(0.1)
    robot.set_motors(left_motor, right_motor)
    time.sleep(0.2)
    robot.set_motors(left_motor/2, right_motor/2)
    time.sleep(0.05)
    robot.stop()

## Data Collection

Let's start data collection. By clicking on somewhere on the camera feed, a snapshot will be taken and the robot will try to move toward the clicked point, i.e. target point. The x and y coordinates of the target point will be saved as a part of the filename.  

In [7]:
import numpy as np
import cv2
import glob
import ipywidgets
from jupyter_clickable_image_widget import ClickableImageWidget
from datetime import datetime

camera = Camera()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
count_widget = ipywidgets.IntText(description='count')
# manually update counts at initialization
count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save data
        now = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{now}_xy_{x}_{y}_.jpg"
        image_path = os.path.join(DATASET_DIR, filename)
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        
        # move robot
        move_robot(x, y)
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    count_widget])

display(data_collection_widget)

## Stop Camera
After you finished the data collection, stop the camera.

In [8]:
camera.stop()

## Zip Dataset
Make `dataset_reg` directory to `dataset_reg.zip` so that we can upload it to Google Colab for training a model.

In [9]:
!zip -r -q dataset_reg.zip dataset_reg


---

## ***(Danger!!) Delete All Dataset***

Next commands delete all of your collected data in the ``dataset_reg`` directory.

In [ ]:
# import shutil
# shutil.rmtree("dataset_reg")